In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

print(tf.__version__)

2.16.1


In [1]:
import pickle
with open('Models\label_encoder_new.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

c:\Users\ABHISHEK\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import mediapipe as mp

model = load_model('Models\\best_model (7).keras')
mp_pose = mp.solutions.pose
mp_face_mesh = mp.solutions.face_mesh
mp_hands = mp.solutions.hands

COORDINATES = 3  
filtered_hand = list(range(21))
filtered_pose = [0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
filtered_face = [10, 152, 14, 7, 8, 144, 145, 146, 153, 154, 155, 158, 159, 160, 161, 163, 362, 385, 387, 388, 390, 398, 400, 402, 417, 419, 420, 421, 423, 424, 425, 426, 427, 428, 429, 430, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449]
POSE_LANDMARKS = len(filtered_pose)
FACE_LANDMARKS = len(filtered_face)
HAND_LANDMARKS = len(filtered_hand)
TOTAL_LANDMARKS = POSE_LANDMARKS + FACE_LANDMARKS + (2 * HAND_LANDMARKS)

def pad_landmarks(landmarks, max_landmarks):
    """Pad landmarks with zeros to ensure consistent shapes."""
    if landmarks.shape[0] < max_landmarks:
        padding = np.zeros((max_landmarks - landmarks.shape[0], COORDINATES))
        landmarks = np.vstack((landmarks, padding))
    return landmarks

def extract_pose_landmarks(frame, pose):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)
    if results.pose_landmarks:
        landmarks = np.array([[lm.x, lm.y, lm.z] for i, lm in enumerate(results.pose_landmarks.landmark) if i in filtered_pose])
        return pad_landmarks(landmarks, POSE_LANDMARKS)
    return np.zeros((POSE_LANDMARKS, COORDINATES))

def extract_face_landmarks(frame, face_mesh):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    if results.multi_face_landmarks:
        landmarks = np.array([[lm.x, lm.y, lm.z] for face_landmarks in results.multi_face_landmarks for i, lm in enumerate(face_landmarks.landmark) if i in filtered_face])
        return pad_landmarks(landmarks, FACE_LANDMARKS)
    return np.zeros((FACE_LANDMARKS, COORDINATES))

def extract_hand_landmarks(frame, hands):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        landmarks = []
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks.extend([[lm.x, lm.y, lm.z] for i, lm in enumerate(hand_landmarks.landmark) if i in filtered_hand])
        return pad_landmarks(np.array(landmarks), 2 * HAND_LANDMARKS)
    return np.zeros((2 * HAND_LANDMARKS, COORDINATES))

def process_video(video_path):
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, 
                                max_num_faces=1, 
                                min_detection_confidence=0.5, 
                                min_tracking_confidence=0.5)
    hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        pose_landmarks = extract_pose_landmarks(frame, pose)
        face_landmarks = extract_face_landmarks(frame, face_mesh)
        hand_landmarks = extract_hand_landmarks(frame, hands)

        landmarks = np.concatenate((pose_landmarks, face_landmarks, hand_landmarks), axis=0)
        if landmarks.shape!= (TOTAL_LANDMARKS, COORDINATES):
            print(f"Warning: Unexpected landmarks shape {landmarks.shape} for video {video_path}")
            landmarks = np.zeros((TOTAL_LANDMARKS, COORDINATES))

        frames.append(landmarks)

    cap.release()

    frames = np.array(frames)

    prediction = model.predict(frames)
    predicted_class = np.argmax(prediction, axis=1)

    predicted_labels = label_encoder.inverse_transform(predicted_class)

    from collections import Counter
    most_frequent_label = Counter(predicted_labels).most_common(1)[0][0]

    return most_frequent_label

In [15]:
video_path = 'test\good_evening.mp4'
predicted_label = process_video(video_path)
print(predicted_label)

c:\Users\ABHISHEK\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
dry


In [ ]:
import os
import random
video_dir = '/kaggle/input/include/'
all_videos = []
for root, _, files in os.walk(video_dir):
    for file in files:
        if file.endswith(('.mp4', '.mov', '.avi', '.MOV')):  # Modify extensions as needed
            all_videos.append(os.path.join(root, file))

# Randomly select 25 videos
random_videos = random.sample(all_videos, 25)

def get_expected_label(file_path):
    label = os.path.basename(os.path.dirname(file_path)).lower()
    return label.split('.')[1].strip()

# Initialize counters
correct_predictions = 0
for video_path in random_videos:
    predicted_label = process_video(video_path).lower()
    print(predicted_label)
    expected_label = get_expected_label(video_path)
    print(expected_label)

    if predicted_label == expected_label:
        correct_predictions += 1

accuracy = (correct_predictions / 25) * 100
print(f'Accuracy: {accuracy}%')

W0000 00:00:1730400265.047234     608 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400265.078017     607 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400265.123984     613 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400265.174178     611 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400265.265952     604 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400265.331233     604 inference_feedback_manager.cc:114] Feedback manager 

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
gift
gift


W0000 00:00:1730400272.629979     632 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400272.681847     632 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400272.727489     638 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400272.777276     638 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400272.840411     628 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400272.908185     628 inference_feedback_manager.cc:114] Feedback manager 

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
white
white


W0000 00:00:1730400280.158195     656 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400280.203512     656 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400280.249741     659 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400280.317344     662 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400280.410112     651 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400280.475547     651 inference_feedback_manager.cc:114] Feedback manager 

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
healthy
healthy


W0000 00:00:1730400464.832387    1160 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400464.870378    1160 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400464.933597    1165 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400464.984900    1164 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400465.052966    1156 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400465.123069    1157 inference_feedback_manager.cc:114] Feedback manager 

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
short
short


W0000 00:00:1730400472.268459    1185 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400472.310239    1184 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400472.325563    1188 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400472.375530    1191 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400472.485974    1180 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730400472.554257    1180 inference_feedback_manager.cc:114] Feedback manager 

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
good morning
good morning
Accuracy: 100.0%
